#### Bed allocation

To follow the patient arrival step, Perplexity was used to generate code that could take the CSV output from the patient arrivals step and allocate the patients to beds according to the rules as set out in the Penn et al. (2020) paper.  To design the code, Perplexity was provided with the following prompt:

**Allocation of patients to the ward – Perplexity Prompt**

*Main command* 

Code a discrete-event simulation model of a ward in python 3.10 and simpy 4. 

Please code the full model specified. Do not return a simplified version. Show all code. 

*General model logic* 

All time units in the model are in hours. 

Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1. 

Patients, provided in the attached CSV file, arrive at the ward and must be allocated a bed in the ward.  The following parameters apply: 

There is 1 ward with 5 bays, each with 3 beds in. 

All people should be allocated to a bed in a ward with the same gender.  The beds should be allocated according to this prioritization of steps: 

 
 1  If there are spaces in at least bay containing patients of the same gender as the patient to be assigned, then assign the patient to the one with the heist proportion of its beds occupied.  Bay index number is used as a tie break if required. 

 2  If the patient was not assigned a bed in step 1 and there are any empty bays assign the patient to the bay with the heist index number. 

 3  If a patient was not assigned a bed in step 1 or 2 and there are any spaces in single rooms available assign the patient to a single room. 

 4  If the patient was not assigned a bed in step 1, 2 or 3 and there is a patient of the opposite gender in a single room and it is possible to move that patient to an appropriately genered bay move that patient and assign the original patient to the single room. 

 5  Else assign a short waiting time to the patient and send them back to the queue. 


Produce a CSV output file of the waiting times, and any patient queue. 


All patient types must have their own generator function 

The model should print out useful information after each event. 

The model should include a user settable run length. This defaults to 12 months. 


Simulation methodology 

numpy should be use for sampling. 

Each interarrival distribution should have its own numpy.random.Generator object 



In [1]:
import simpy
import numpy as np
import csv
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Dict, Tuple

@dataclass
class Patient:
    id: int
    arrival_time: float
    gender: str
    los: float

class CCU:
    def __init__(self, env: simpy.Environment, num_bays: int, beds_per_bay: int):
        self.env = env
        self.num_bays = num_bays
        self.beds_per_bay = beds_per_bay
        self.bays: List[List[Patient]] = [[] for _ in range(num_bays)]
        self.single_rooms: List[Patient] = []
        self.waiting_queue: List[Patient] = []
        self.patient_id = 0
        self.waiting_times: Dict[int, float] = {}
        self.rng = np.random.default_rng()

    def allocate_bed(self, patient: Patient) -> bool:
        # Step 1: Allocate to a bay with same gender patients
        same_gender_bays = [i for i, bay in enumerate(self.bays) if bay and bay[0].gender == patient.gender]
        if same_gender_bays:
            target_bay = max(same_gender_bays, key=lambda i: len(self.bays[i]) / self.beds_per_bay)
            if len(self.bays[target_bay]) < self.beds_per_bay:
                self.bays[target_bay].append(patient)
                return True

        # Step 2: Allocate to an empty bay
        empty_bays = [i for i, bay in enumerate(self.bays) if not bay]
        if empty_bays:
            self.bays[max(empty_bays)].append(patient)
            return True

        # Step 3: Allocate to a single room
        if len(self.single_rooms) < 1:
            self.single_rooms.append(patient)
            return True

        # Step 4: Try to move opposite gender patient from single room to bay
        if self.single_rooms and self.single_rooms[0].gender != patient.gender:
            opposite_patient = self.single_rooms[0]
            for i, bay in enumerate(self.bays):
                if bay and bay[0].gender == opposite_patient.gender and len(bay) < self.beds_per_bay:
                    self.bays[i].append(opposite_patient)
                    self.single_rooms[0] = patient
                    return True

        # Step 5: Send back to queue
        return False

    def admit_patient(self, patient: Patient):
        if self.allocate_bed(patient):
            print(f"Time {self.env.now:.2f}: Patient {patient.id} ({patient.gender}) admitted")
            self.env.process(self.patient_stay(patient))
        else:
            print(f"Time {self.env.now:.2f}: Patient {patient.id} ({patient.gender}) queued")
            self.waiting_queue.append(patient)
            self.env.process(self.queue_patient(patient))

    def queue_patient(self, patient: Patient):
        wait_time = self.rng.exponential(1)  # Short waiting time, mean of 1 hour
        yield self.env.timeout(wait_time)
        self.waiting_queue.remove(patient)
        self.waiting_times[patient.id] = wait_time
        self.admit_patient(patient)

    def patient_stay(self, patient: Patient):
        yield self.env.timeout(patient.los)
        self.discharge_patient(patient)

    def discharge_patient(self, patient: Patient):
        for bay in self.bays:
            if patient in bay:
                bay.remove(patient)
                break
        else:
            if patient in self.single_rooms:
                self.single_rooms.remove(patient)

        print(f"Time {self.env.now:.2f}: Patient {patient.id} discharged")

        if self.waiting_queue:
            next_patient = self.waiting_queue.pop(0)
            self.admit_patient(next_patient)

def patient_generator(env: simpy.Environment, ccu: CCU, csv_file: str):
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            yield env.timeout(float(row['arrival_time']))
            patient = Patient(
                id=ccu.patient_id,
                arrival_time=env.now,
                gender=row['Gender'],
                los=float(row['LoS'])
            )
            ccu.patient_id += 1
            ccu.admit_patient(patient)

def run_simulation(csv_file: str, run_length: float = 8760):  # Default 12 months (365 days * 24 hours)
    env = simpy.Environment()
    ccu = CCU(env, num_bays=5, beds_per_bay=3)
    env.process(patient_generator(env, ccu, csv_file))
    env.run(until=run_length)

    # Output waiting times
    with open('waiting_times.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Patient_ID', 'Waiting_Time'])
        for patient_id, wait_time in ccu.waiting_times.items():
            writer.writerow([patient_id, wait_time])

    # Output final queue
    with open('final_queue.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Patient_ID', 'Gender', 'Arrival_Time'])
        for patient in ccu.waiting_queue:
            writer.writerow([patient.id, patient.gender, patient.arrival_time])

if __name__ == "__main__":
    run_simulation('hospital_patients.csv')


Time 4.30: Patient 0 (female) admitted
Time 18.01: Patient 1 (female) admitted
Time 72.63: Patient 2 (female) admitted
Time 124.30: Patient 0 discharged
Time 144.63: Patient 2 discharged
Time 186.01: Patient 1 discharged
Time 199.17: Patient 3 (female) admitted
Time 319.17: Patient 3 discharged
Time 378.87: Patient 4 (male) admitted
Time 450.87: Patient 4 discharged
Time 578.53: Patient 5 (female) admitted
Time 746.53: Patient 5 discharged
Time 826.46: Patient 6 (female) admitted
Time 946.46: Patient 6 discharged
Time 1132.34: Patient 7 (female) admitted
Time 1204.34: Patient 7 discharged
Time 1516.58: Patient 8 (male) admitted
Time 1588.58: Patient 8 discharged
Time 1915.48: Patient 9 (female) admitted
Time 2035.48: Patient 9 discharged
Time 2327.75: Patient 10 (female) admitted
Time 2495.75: Patient 10 discharged
Time 2785.95: Patient 11 (female) admitted
Time 2857.95: Patient 11 discharged
Time 3345.25: Patient 12 (female) admitted
Time 3465.25: Patient 12 discharged
Time 3908.46: P